# Lab Class ML:V

## Ankit Satpute, 120825 ; Hsueh Wei, 120820; Sagar Nagaraj Simha, 120797 - (M.Sc. CS4DM) - Group 13

## Exercise 5 - Perceptron Training

In [134]:
from sklearn.datasets import make_classification
import numpy as np
import random
import pandas as pd

In [135]:
def separable_dataset():
    X, C = make_classification(300, 2, 2, 0, 0, 2, 1,
                                class_sep=30, flip_y=0,
                                hypercube=False, random_state=2)
    X = np.hstack([np.ones((X.shape[0], 1)), X])
    C = 2*C - 1
    D = (X, C)
    return D

In [136]:
D = separable_dataset() 

In [137]:
#D[0] is X matrix of size nx3
#D[1] is C matrix of size nx1

In [138]:
D[0][0] # First sample in X

array([1.        , 3.94931935, 6.85201923])

In [139]:
D[1][0] #First sample in C

1

## 3(a)

In [140]:
def random_select(D):
    # generates a random integer number between 0 and len(D)-1, inclusive
    i = random.randint(0, len(D[0])-1)
    #print(i)
    # returns the tuple of index i in D 
    return ((D[0][i],D[1][i]))

In [141]:
random_select(D)

(array([ 1.        , -6.72592941,  6.94875964]), -1)

In [142]:
#Test case
random_select(([[1,0,0]], [-1]))

([1, 0, 0], -1)

## 3(b)

In [143]:
def convergence(D, w):
    #Picks X and C from D
    X = D[0]
    C = D[1] 
    y = np.sign(np.dot(X,w)) #wT.x
    equal = np.array_equal(y, C) #Returns True if two numpy arrays are equal, False otherwise 
    return equal

In [144]:
#Test cases
X_t = np.array([[1, 0, 0]])
C_t = np.array([1])
w_t = np.array([1, 1, 1])

In [145]:
convergence(D=(X_t, C_t), w=w_t)

True

In [146]:
convergence(D=(X_t, C_t), w=-w_t)

False

## 3(c)

In [147]:
def pt(D,eta, tmax=300):
    # Perceptron Training Algorithm
    t = 0
    delta_w = 0
    #Initialize weights with zeros
    #w = np.zeros(3)
    #Initialize weights with a value between 0 and 1 drawn from an uniform distribution
    w = np.array([random.uniform(0, 1),random.uniform(0, 1),random.uniform(0, 1)])
    #Repeat until convergence or max iterations
    while( (not convergence(D,w)) and t<=tmax):
        t+=1
        (x,c) = random_select(D)
        error = c - np.sign(np.dot(x,w))
        delta_w = eta * error * x
        w+=delta_w
    return w # return weight vector

## 3(d)

In [148]:
def misclass_rate(D, w):
    # Misclassification rate is calculated as (number of wrong classifications / Total number of samples)
    X = D[0]
    C = D[1] 
    y = np.sign(np.dot(X,w)) #threshold(wT.x)
    comparison = (C==y) #Returns an array with True/False for element wise equality 
    missclassification_rate = (np.size(comparison) - np.count_nonzero(comparison)) / np.size(comparison) #Counts number of False values 
    return missclassification_rate

In [149]:
#Test case
D = separable_dataset()
w = pt(D, eta=0.1, tmax=300)
print('Missclassification rate is', misclass_rate(D, w))

Missclassification rate is 0.0


## 3(e)

In [150]:
def nonseparable_dataset():
    X, C = make_classification(300, 2, 2, 0, 0, 2, 1,
                                class_sep=10, flip_y=0.1,
                                hypercube=False, random_state=2)
    X = np.hstack([np.ones((X.shape[0], 1)), X])
    C = 2*C - 1
    return (X, C)

In [151]:
missclassification_rate_ = []
#Performs for 1000 runs
print('Running PT for 1000 runs')
for i in range(1000):
    D = nonseparable_dataset()
    w = pt(D, eta=0.1, tmax=300)
    missclassification_rate_.append(round(misclass_rate(D, w),3))
    print('.',end='')
    #print('run:',i, '::misclassification rate:', round(misclass_rate(D, w),3))
print('\nPT for 1000 runs completed')

Running PT for 1000 runs
...............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [152]:
missclassification_rate_s = pd.Series(missclassification_rate_)
stats = missclassification_rate_s.describe()
print('Misclasssification rate statistics - PT:', '\n', 'Minimum - ', stats['min'], '\n', 'Average - ', round(stats['mean'],3), '\n', 'Maximum - ', stats['max'])

Misclasssification rate statistics - PT: 
 Minimum -  0.07 
 Average -  0.139 
 Maximum -  0.513


In [153]:
#missclassification_rate_s.plot.hist()

## 3(f)

In [154]:
# Batch gradient descent

In [155]:
def bgd(D,eta, tmax=300):
    t = 0
    #Initialize weights with zeros
    #w = np.zeros(3)
    #Initialize weights with a value between 0 and 1 drawn from an uniform distribution
    w = np.array([random.uniform(0, 1),random.uniform(0, 1),random.uniform(0, 1)])
    #Repeat until convergence or max iterations
    while( (not convergence(D,w)) and t<=tmax):
        t+=1
        delta_w = 0
        for k in range(len(D[0])):
            x = D[0][k]
            c = D[1][k]
            error = c - np.sign(np.dot(x,w))
            delta_w+= eta * error * x
        w+=delta_w
    return w

In [156]:
missclassification_rate_bgd = []
# For 1000 runs
print('Running BGD for 1000 runs')
for i in range(1000):
    D_ = nonseparable_dataset()
    w_ = bgd(D_, eta=0.01, tmax=300)
    #print(w_)
    missclassification_rate_bgd.append(round(misclass_rate(D_, w_),3))
    print('.',end='')
    #print('run:',i, '::misclassification rate BGD:', round(misclass_rate(D_, w_),3))
print('\nBGD for 1000 runs completed')

Running BGD for 1000 runs
..............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [157]:
missclassification_rate_s_bgd = pd.Series(missclassification_rate_bgd)
#missclassification_rate_s
stats_bgd = missclassification_rate_s_bgd.describe()
print('Misclasssification rate statistics - BGD:', '\n', 'Minimum - ', stats_bgd['min'], '\n', 'Average - ', stats_bgd['mean'], '\n', 'Maximum - ', stats_bgd['max'])

Misclasssification rate statistics - BGD: 
 Minimum -  0.07 
 Average -  0.13953200000000002 
 Maximum -  0.487


In [158]:
#missclassification_rate_s_bgd.plot.hist()

## Comparison -  
### 1) The performance seem to be strongly dependent on how the weight vectors are initialized. 
In case of a weights chosen from a uniform random number between 0 and 1,

Misclasssification rate statistics - PT: 
 Minimum -  0.07,
 Average -  0.139, 
 Maximum -  0.513.
Misclasssification rate statistics - BGD: 
 Minimum -  0.07, 
 Average -  0.13953200000000002, 
 Maximum -  0.487.

We can observe that the minimum and average misclassification rate is similar but in terms of the maximum value, BGD performs better than Pt with a lower misclassification rate.  


In case of a weights chosen as a zero vector [0,0,0] ,
Misclasssification rate statistics - PT: 
 Minimum -  0.07, 
 Average -  0.137, 
 Maximum -  0.593.
Misclasssification rate statistics - BGD: 
 Minimum -  0.08333, 
 Average -  0.83333333, 
 Maximum -  0.083333.
 
Here, we observe that the BGD algorithm saturates to 0.083 throught all the runs since each time the weight vectors are initialized to zero and the algorithm goes through each sample.
  
### 2) A histogram analysis shows that the BGD has lower variance from the mean compared to Pt
### 3) The BGD algorithm takes a longer time compared to Pt since it goes through all the training data in each iteration while Pt goes through a random sample in each iteration